In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize importing libraries

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

import numpy as np
import pandas as pd
from google.colab import files
from tqdm import tqdm
import warnings
warnings.simplefilter('ignore')
import gc
from scipy.special import softmax
from simpletransformers.classification import ClassificationModel
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold 
import sklearn
from sklearn.metrics import log_loss
from sklearn.metrics import *
from sklearn.model_selection import *
import re
import random
import torch
pd.options.display.max_colwidth = 200

#choose the same seed to assure that our model will be roproducible

def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
        torch.backends.cudnn.benchmark = False

seed_all(2)

In [ ]:
cd '/content/drive/My Drive/sql-jing1'

/content/drive/My Drive/sql-jing1


In [ ]:
import random
from transformers import BertTokenizer, BertModel
Y = ['mountain', 'border', 'high', 'city', 'state', 'river']
def read_file(data='train',batch_size=16,max_len=22):

  filename = data + '_org.qu'
  x, xt, y = [], [], []
  sent_x, sent_t = [], []

  with open(filename) as f:
    ls = f.readlines()
  total = len(ls)
  num_batch = int((total+batch_size)/batch_size)
  for i in range(num_batch):
    st = batch_size * i
    ed = min(batch_size * i + batch_size, total)
    x_batch, xt_batch, y_batch = [], [], []
    sent_x_batch, sent_t_batch = [], []
    for j in range(st, ed):
      l = ls[j]
      tables = l.strip('\n').split('\t')[0].split()
      tables = [t.replace('table_','').replace('_info','') for t in tables]
      
      question = l.strip('\n').split('\t')[1]
      tokenized_input = tokenizer.tokenize(question) 
      while len(tokenized_input) < max_len:
          tokenized_input.append('[PAD]')
      x_idx = tokenizer.convert_tokens_to_ids(tokenized_input)
      #positive samples
      for t in tables:
        sent_x_batch.append(question)
        #print(x_idx)
        x_batch.append(x_idx)

        sent_t_batch.append(t)
        xt_idx = tokenizer.convert_tokens_to_ids([t])
        xt_batch.append(xt_idx)
        y_batch.append(1.0)

      #negative sampling
      YY = [t for t in Y if t not in tables]
      for t in YY:
        sent_x_batch.append(question)
        sent_t_batch.append(t)
        xt_idx = tokenizer.convert_tokens_to_ids([t])
        x_batch.append(x_idx)
        xt_batch.append(xt_idx)
        y_batch.append(0.0)

      #positive sampling for balance
      if data == '.':
        for j in range(len(YY)-len(tables)):
          t = random.choice(tables)
          sent_x_batch.append(question)
          sent_t_batch.append(t)
          x_batch.append(x_idx)
          xt_idx = tokenizer.convert_tokens_to_ids([t])
          xt_batch.append(xt_idx)
          y_batch.append(1.0)

    x_batch = np.asarray(x_batch)
    #print(x_batch.shape)
    x_batch = np.transpose(x_batch, (1,0))
    xt_batch = np.asarray(xt_batch)
    xt_batch = np.transpose(xt_batch, (1,0))
    y_batch = np.asarray(y_batch)
    sent_x_batch = np.asarray(sent_x_batch)
    sent_t_batch = np.asarray(sent_t_batch)

    x.append(torch.tensor(x_batch, dtype=torch.long, device=device))
    xt.append(torch.tensor(xt_batch, dtype=torch.long, device=device))
    y.append(torch.tensor(y_batch, dtype=torch.float, device=device))
    sent_x.append(sent_x_batch)
    sent_t.append(sent_t_batch)
  print(total)
  return x, xt, y, sent_x, sent_t

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_x, train_xt, train_y, _, _ = read_file('train')
assert len(train_x) == len(train_xt) == len(train_y)
test_x, test_xt, test_y, sent_x, sent_t = read_file('test')
assert len(test_x) == len(test_xt) == len(test_y) == len(sent_x) == len(sent_t)




1647
279


In [ ]:
pos, ttl = 0, 0
for l in train_y:
  ttl += l.size(0)
  pos += sum(l)
print('train positive {0} total {1}'.format(pos, ttl))
pos, ttl = 0, 0
for l in test_y:
  ttl += l.size(0)
  pos += sum(l)
print('test positive {0} total {1}'.format(pos, ttl))

train positive 1890.0 total 9897
test positive 329.0 total 1674


In [ ]:
class Encoder(nn.Module):
    def __init__(self, emb, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        print('New bert embedding')
        #bertmodel = BertModel.from_pretrained('bert-base-uncased')
        #bertmodel.resize_token_embeddings(input_dim)
        #self.embedding = bertmodel.embeddings
        self.embedding = emb
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)

class Decoder(nn.Module):
    def __init__(self, emb, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.attention = attention
        
        self.embedding = emb
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, 1)
        #self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, 1)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        #input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]

        #print(embedded.size())
        #print(weighted.size())
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
  
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        #prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))

        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
          super().__init__()
        
          self.encoder = encoder
          self.decoder = decoder
          self.device = device
          self.output_layer = nn.Sigmoid()

    def forward(self, src, trg):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg
        #input =  torch.tensor([101]*BATCH_SIZE, device=device)
          
        #insert input token embedding, previous hidden state and all encoder hidden states
        #receive output tensor (predictions) and new hidden state
        output, hidden = self.decoder(input, hidden, encoder_outputs)
        output = output.squeeze(1)
        output = self.output_layer(output)
          
        return output

In [ ]:
INPUT_DIM = len(tokenizer)
ENC_EMB_DIM = 768 
DEC_EMB_DIM = 768 
ENC_HID_DIM = 256
DEC_HID_DIM = 256
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

print(INPUT_DIM)

bertmodel = BertModel.from_pretrained('bert-base-uncased')
emb = bertmodel.embeddings
#emb = nn.Embedding(INPUT_DIM, ENC_EMB_DIM)
attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(emb, INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(emb, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)
model = Seq2Seq(enc, dec, device).to(device)


30522



New bert embedding


In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (rnn): GRU(768, 256, bidirectional=True)
    (fc): Linear(in_features=512, out_features=256, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=768, out_features=256, bias=True)
      (v): Linear(in_features=256, out_features=1, bias=False)
    )
    (embedding): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=F

In [ ]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCELoss()
def train(model, iterator, optimizer, criterion, clip):
 
    #model.load_state_dict(torch.load('tut3-model.pt'))
    model.train()

    epoch_loss = 0  
    acc = 0
    total = 0
    right = 0
    num_iter = len(train_x)
    for i in range(num_iter):
    #for i, batch in enumerate(iterator):
        #print('{0}/{1}'.format(i,len(train_input), end='\r'))
        src = train_x[i]
        trg = train_xt[i]
        #src = torch.transpose(src, 1, 0)
        #trg = torch.transpose(trg, 1, 0)
        #src.transpose_(0,1)
        #trg.transpose_(0,1)
        
        #src = batch.src
        #trg = batch.trg

        optimizer.zero_grad()
        
        output = model(src, trg)
         
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        #output_dim = output.shape[-1]
        
        #output = output[1:].view(-1, output_dim)

        #trg = trg[1:].view(-1)
        #trg = trg[1:].reshape((MAX_LENGTH-1)*trg[1:].size(1))

        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, train_y[i])
        #print(output)
        #print(train_y[i])
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()

        y_pred = np.array(output.cpu().detach())
        y_pred = np.where(y_pred <= 0.5, 0, y_pred)
        y_pred = np.where(y_pred > 0.5, 1, y_pred)
        ybar = train_y[i].cpu().detach().numpy()
        acc += sum(y_pred==ybar)
        total += len(ybar)
    
   
    
    return epoch_loss / num_iter, acc*1.0 / total

In [ ]:
def evaluate(model, iterator, criterion, PRINT=False):
    
    w = open('test_org.qu.new', 'w')
    model.eval()
    epoch_loss = 0
    acc = 0
    total = 0
    wr = 0
    pre = ''
    ts = ''
    with torch.no_grad():
        num_iter = len(test_x)
        for i in range(num_iter):
        #for i, batch in enumerate(iterator):
            src = test_x[i]
            trg = test_xt[i]
            #src = torch.transpose(src, 1, 0)
            #trg = torch.transpose(trg, 1, 0)
            
            output = model(src, trg) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            #output_dim = output.shape[-1]
            
            #output = output[1:].view(-1, output_dim)
            #trg = trg[1:].view(-1)
            #trg = trg[1:].reshape((MAX_LENGTH-1)*trg[1:].size(1))

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, test_y[i])
            y_pred = np.array(output.cpu().detach())
            y_pred = np.where(y_pred <= 0.5, 0.0, y_pred)
            y_pred = np.where(y_pred > 0.5, 1.0, y_pred)
            ybar = test_y[i].cpu().detach().numpy()
            acc += sum(y_pred==ybar)
            total += len(ybar)
            epoch_loss += loss.item()
            assert len(sent_x[i]) == len(y_pred) == len(ybar) == len(sent_t[i])
            if PRINT:
              for j in range(len(sent_x[i])):
                #print('---------')
                #print(sent_x[i][j])
                #print(sent_t[i][j])
                #print(y_pred[j])
                #print(ybar[j])
                if pre == '':
                  pre = sent_x[i][j]
                if pre != '' and pre != sent_x[i][j]:
                  w.write(ts.strip() + '\t' + pre + '\n')
                  pre = sent_x[i][j]
                  ts = ''
                t = sent_t[i][j]
                t = t.replace('border','border_info')
                t = t.replace('high','highlow')
                t = 'table_' + t
                if y_pred[j]==1.0:
                  ts += (t + ' ') 
        w.write(ts.strip() + '\t' + pre)    
    w.close()  
    print(f'\t Val. Loss: {epoch_loss / num_iter:.3f} Test Acc: {acc*1.0 / total:.3f}')      
        
    return epoch_loss / num_iter, acc*1.0 / total

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [ ]:
import time
N_EPOCHS = 30
CLIP = 5

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()  
    train_loss, train_acc = train(model, None, optimizer, criterion, CLIP)
    valid_loss, valid_acc = evaluate(model, None, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} Train Acc: {train_acc:.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} Test Acc: {valid_acc:.3f}')

In [ ]:
model.load_state_dict(torch.load('tut3-model-997.pt'))
valid_loss, valid_acc = evaluate(model, None, criterion, PRINT=True)

	 Val. Loss: 0.021 Test Acc: 0.997


In [ ]:
wr_arr = ['4', '24', '31', '34', '45', '57', '63', '83', '88', '91', '92', '105', '108', '109', '110', '118', '119', '120', '122', '123', '127', '129', '130', '137', '139', '140', '141', '142', '144', '157', '158', '159', '163', '166', '171', '175', '176', '177', '178', '179', '180', '181', '183', '186', '195', '196', '197', '200', '201', '202', '203', '205', '212', '213', '214', '216', '218', '220', '222', '223', '224', '226', '229', '230', '235', '236', '237', '238', '240', '241', '242', '243', '244', '245', '246', '247', '250', '251', '252', '253', '254', '255', '256', '257', '258', '259', '260', '261', '262', '264', '266', '267', '270', '271', '272', '274', '276']
with open('test_org.qu', 'r') as f1, open('test_org.qu.new', 'r') as f2:
  ls1 = f1.readlines()
  ls2 = f2.readlines()

cnt = 0
i = 0
for l1, l2 in zip(ls1, ls2):
  if l1 != l2:
    i += 1
    if str(i) in wr_arr:
      pass
    else:
      cnt += 1
    print(l1.strip())
    print(l2.strip())
    print('====')

print(1-len(wr_arr)/len(ls1)*1.0)
print(cnt/len(ls1)*1.0)
print(1-len(wr_arr)/len(ls1)*1.0-cnt/len(ls1)*1.0)

table_highlow table_state table_city	how many states have a higher point than the highest point of the state with the largest capital city in the us
table_highlow table_state	how many states have a higher point than the highest point of the state with the largest capital city in the us
====
table_city	what is the smallest city in the us
table_city table_state	what is the smallest city in the us
====
table_border_info table_city	what states border the state with the most cities
table_border_info table_city table_state	what states border the state with the most cities
====
table_mountain	which is the highest peak not in alaska
table_mountain table_border_info	which is the highest peak not in alaska
====
table_river table_border_info table_state	which rivers run through states that border the state with the capital austin
table_river table_border_info	which rivers run through states that border the state with the capital austin
====
0.6523297491039426
0.014336917562724014
0.63799283154121